In [1]:
#необходимые пакеты 
import ee, geemap, os
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import geopandas as gpd 
import scipy

In [2]:
ee.Initialize()

### Границы региона 

In [3]:
region_boundary = geemap.geojson_to_ee('data/budenovsk_district_boundaries.geojson')

Budenovsk = geemap.Map(zoom = 10)

Budenovsk.centerObject(region_boundary)

Budenovsk.addLayer(region_boundary ,{'color' : "FF0000"}, 'Границы региона' )

Budenovsk

Map(center=[44.745544055004785, 44.14114333519604], controls=(WidgetControl(options=['position'], widget=HBox(…

### подготовка коллекции для дальнейшей обработки

In [5]:
# возьмем коллекцию сентинел и обработаем ее немного
def masking(img) : 
    cloudProb = img.select('MSK_CLDPRB')  # покрытие облаками
    snowProb = img.select('MSK_SNWPRB') # покрытие снегом
    cloud = cloudProb.lt(1) # создали бинарную маску иными словами просто все что имеет значение меньше 5 одна группа выше другая
                            # а мы помним что пиксели принимают значения от 0 до 255
    snow = snowProb.lt(1) # тоже самое что с облаками
    scl = img.select('SCL') # слой с классификатором(есть в sentinel 2 уровня обработки 2А)
    shadow = scl.neq(3);# 3 в классификации это тени от облаков
    cirrus_medium = scl.neq(8) # тоже по классификации облака 
    cirrus_high = scl.neq(9) # аналогично облака
    cirrus = scl.neq(10); # 10 это перистые облака или цирусы
    masked_img = img.updateMask(cloud).updateMask(shadow).updateMask(cirrus).updateMask(cirrus_medium).updateMask(cirrus_high)
    return(masked_img)

def clipper_region(image):
    clipped = image.clip(region_boundary.geometry())
    return  clipped 

#создали коллекцию с изображениями
start = ee.Date('2018-09-01')
finish = ee.Date('2019-09-01')
sentinel2_collection = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterBounds(region_boundary.geometry()) \
    .filterMetadata("CLOUD_COVERAGE_ASSESSMENT", 'less_than', 20.0) \
    .filterMetadata('SNOW_ICE_PERCENTAGE','less_than', 5.0) \
    .filterDate(start, finish)  \
    .map(masking).map(clipper_region)
sentinel2_collection_list = sentinel2_collection.toList(sentinel2_collection.size().getInfo())

#вынимаем уникальные даты из датасета
sentinel2_collection_list = sentinel2_collection.toList(sentinel2_collection.size().getInfo())
sentinel2_time_list = []
for i in range(sentinel2_collection.size().getInfo()):
    img = ee.Image(sentinel2_collection_list.get(i))
    time = ee.Date(img.get('system:time_start')).format('YYYY-MM-dd').getInfo()
    sentinel2_time_list.append(time)

#уникальные даты из датасета
unique_dates = sorted(list(set(sentinel2_time_list)))

diff = finish.difference(start , 'day')
Range = ee.List.sequence(0, diff.subtract(1)).map(lambda day :  start.advance(day,'day'))
def day_mosaics(date , newlist):
    date = ee.Date(date)
    newlist = ee.List(newlist)
    
    filtered = sentinel2_collection.filterDate(date , date.advance(1,'day'))
    
    image = ee.Image(filtered.mosaic())
    
    return ee.List(ee.Algorithms.If(filtered.size(), newlist.add(image), newlist))
sentinel2_collection_mosaic = ee.ImageCollection(ee.List(Range.iterate(day_mosaics, ee.List([]))))

In [6]:
sentinel2_collection_mosaic.getInfo()

{'type': 'ImageCollection',
 'bands': [],
 'features': [{'type': 'Image',
   'bands': [{'id': 'B1',
     'data_type': {'type': 'PixelType',
      'precision': 'int',
      'min': 0,
      'max': 65535},
     'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0]},
    {'id': 'B2',
     'data_type': {'type': 'PixelType',
      'precision': 'int',
      'min': 0,
      'max': 65535},
     'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0]},
    {'id': 'B3',
     'data_type': {'type': 'PixelType',
      'precision': 'int',
      'min': 0,
      'max': 65535},
     'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0]},
    {'id': 'B4',
     'data_type': {'type': 'PixelType',
      'precision': 'int',
      'min': 0,
      'max': 65535},
     'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0]},
    {'id': 'B5',
     'data_type': {'type': 'PixelType',
      'precision': 'int',
      'min': 0,
      'max': 65535},
     'crs': 'EPSG:4326',
     'crs_transform'

In [21]:
visualization = {"min": 0.0, "max": 2000,"bands": ['B4', 'B3', 'B2']}

коллекция нормальная получилась, завтра продолжим примерный план того чего надо сделать 
1) построить NDTI 
2) добавить геометрии хозяйств(не забыть бы дома все взять что нужно) 
3) вытащить данные(желательно не одну точку а просто тупо вынуть все значения пикселей по геометрии, пускай там будет массив) 
4) проверить нормальность распределения этого массива 
5) выбрать описательную стату(скорее всего это медиана будет) 
6) можно ради интереса построить динамику изменения во времени NDTI (можно еще NDVI) и понять когда значения падают но я не вижу в этом никакого смысла
7) далее надо посчитать minNDTI и уже его значения обсчитать
8) ну и по итогу посмотреть будет ли разница по minNDTI по хозяйствам (ну такой уровень довольно высокий но так я покажу что minNDTI является хорошим прямым признаком прямого посева) 
9) ну и дальше A/B тестирование и визуализация графичками, должно получиться неплохо 


In [22]:
for i in range(23):
    Budenovsk.addLayer(ee.Image(sentinel2_collection_mosaic.toList(23).get(i) ), visualization, unique_dates[i])


